In [ ]:
import pickle
import itertools as it
import numpy as np
import pandas as pd
import networkx as nx
import os
import subprocess as sp
from pathlib import Path
import copy
import re

In [ ]:
categories = [
    0,
    1,
    1,
    1,
    1,
    1,
    3,
    2,
    2,
    3,
    1,
    1,
    2,
    2,
    2,
    3,
    2,
    2,
    2,
    2,
    2,
    3,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    3,
    3,
    2,
    2,
    1,
    1,
    2,
    2,
    2,
    1,
    1,
    2,
    1,
    2,
    2,
    1,
    2,
    2,
    1,
    2,
    2,
    2,
    1,
    1,
    1,
    1,
    2,
    2,
    2,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    2,
    1,
    1,
    2,
    3,
    2,
    2,
    2,
    2,
    2,
    1,
    2,
    2,
    3,
    2,
    2,
    2,
    2,
    1,
    2,
    2,
    1,
    2,
    4,
    1,
    1,
    2,
    2,
    2,
    4,
    2,
    4,
    2,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    4,
    4,
    4,
    4,
    4,
    2,
    4,
    2,
    4,
    2,
    2,
    4,
    4,
    2,
    3,
    3,
    2,
    2,
    2,
    2,
    4,
    3,
    3,
    3,
    3,
    4,
    1,
    3,
    4,
    2
]

In [ ]:
cats = {
    1: "HC",
    2: "FEP",
    3: "Treatment 3+ yr",
    4: "High risk"
}

In [ ]:
from bids import BIDSLayout

layout = BIDSLayout("results/prepdwi_recon", validate=False)
raw_layout = BIDSLayout("../..", validate=False)

In [ ]:
import fury.io as fio
import fury.utils as futil
import fury.lib as flib
import nibabel as nib
def node_sizes(path):
    pld = fio.load_polydata(path)
    cluster_id = flib.numpy_support.vtk_to_numpy(pld.GetCellData().GetArray("parcel_idx"))
    counts = defaultdict(lambda: 0)
    for i in cluster_id:
        counts[i] += 1
    return counts

def node_sizes_relative(path):
    pld = fio.load_polydata(path)
    cluster_id = flib.numpy_support.vtk_to_numpy(pld.GetCellData().GetArray("parcel_idx"))
    counts = [0] * (max(cluster_id)+1)
    for i in cluster_id:
        counts[i] += 1
    spread = max(counts) - min(counts)
    for i in range(len(counts)):
        counts[i] = counts[i]/spread
    return counts


def degree_sizes_relative(degrees):
    spread = max(degrees.values()) - min(degrees.values())
    for i in range(len(degrees)):
        degrees[i] = degrees[i]/spread
    return degrees

def mesh_size(path):
    if Path(path).suffix in [".vtk", ".vtp"]:
        pld = fio.load_polydata(path)
        return len(futil.get_polydata_triangles(pld))
    pld = nib.load(path)
    return len(pld.agg_data('triangle'))

def node_size_at_points(df: pd.DataFrame, points, column):
    df = df.sort_values(column).reset_index(drop=True)
    num_rows = len(df[column])
    return pd.DataFrame({
        column: [df[column][int((num_rows-1) * point)] for point in points],
        "point": points
    })


In [ ]:
def betweenness(b_vals, threshold=.5):
    vals = b_vals.values()
    b_range = max(vals) - min(vals)
    margin = b_range*threshold + min(vals)
    above = dict(filter(lambda val: val[1]>=margin, b_vals.items()))
    # return above
    # b = dict(zip(it.count(), sorted(b_vals)))
    df = pd.DataFrame({"betweenness": above})
    return df

In [ ]:
with open("resources/tract-assignments/hemispheric", 'r') as f:
    data = f.readlines()

all_bundles = [re.search(r"^cluster_(\d+)\.vtp$", s)[1] for s in data]

In [ ]:
bundle_counts = [dict(zip(all_bundles, it.repeat(0))) for _ in range(4)]
major_edges = {}
for bidsfile in layout.get(suffix="connectome", hemi="L"):
    with open(bidsfile.path, 'rb') as f:
        d = pickle.load(f)
    connectome = d["connectome"]
    data = copy.deepcopy(connectome.matrix)
    np.fill_diagonal(data, 0)
    G = nx.from_numpy_matrix(data)
    sub = int(bidsfile.entities['subject'])
    cat = categories[sub]
    for edge in G.edges:
        G.edges[edge]["distance"] = 1/G.edges[edge]["weight"]
    edges = betweenness(nx.edge_betweenness_centrality(G, weight="distance"), threshold=0.9)
    f = {}
    for x in edges:
        p = connectome.get_bundles_of_edge(x)
        total = sum(p.values())
        for k in p:
            frac = p[k] / total
            if frac > 0.1:
                f[k] = frac
    major_edges[sub] = set(f.keys())
    
    bundles = [re.search(r"^cluster_(\d+)$", str(Path(d["bundle_paths"][x]).stem))[1] for x in f.keys()]
    for bundle in bundles:
        bundle_counts[cat-1][bundle] += 1

In [ ]:
import plotly.express as px
df = pd.concat([
    pd.DataFrame({"counts": bundle_counts[x], "category": cats[x+1]}) for x in range(4)
])
fig = px.bar(
    df,
    y="counts",
    color="category",
    labels={
        "index": "Bundle ID"
    },
    title="Most central bundles across subjects",
    width=1178,
    height=400,
)

fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False
)

In [ ]:
tract_assignments = {}
for path in Path("resources/tract-assignments").iterdir():
    if path.name in ["commissural", "hemispheric"]:
        continue
    with path.open('r') as f:
        bundles = [re.search(r"^cluster_(\d+)\.vtp$", s)[1] for s in f.readlines()]
    tract_assignments[path.name] = bundles


In [ ]:
from collections import defaultdict
agg = df.set_index("category", append=True).groupby(level=0).sum()["counts"]
agg = agg.where(agg > 0).dropna()
key_tracts = defaultdict(list)
for bundle in agg.where(agg > agg.quantile(0.7)).dropna().index:
    for name, tract in tract_assignments.items():
        if bundle in tract:
            key_tracts[name].append(bundle)

In [ ]:
dict(zip(tract_assignments["T_Sup-F"], it.count()))["00408"]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

subject = "008"

with open(layout.get(subject=subject, suffix="connectome", hemi="L")[0].path, 'rb') as f:
    connectome = pickle.load(f)
matrix = copy.deepcopy(connectome["connectome"].matrix)
np.fill_diagonal(matrix, 0)
G = nx.from_numpy_matrix(matrix)
for edge in G.edges:
    G.edges[edge]["distance"] = 1/G.edges[edge]["weight"]

df = betweenness(nx.betweenness_centrality(G, weight="distance"), threshold=0)
path = layout.get(subject=subject, suffix="parcellation", extension=".vtk", hemi="L")[0].path

df = pd.concat([
    df,
    node_sizes(path),
    pd.DataFrame({"degree": list(dict(G.degree).values())})
], axis=1)
df = df.sort_values(["betweenness"], ascending=True).reset_index(drop=True)

fig  = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=df.index, y=df["node_size"], mode="markers"),
    secondary_y=True
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["betweenness"], mode="markers"),
    secondary_y=False
)


In [ ]:
raw_layout.get(subject="001", suffix="smoothwm", extension=".surf.gii", hemi="L")

In [ ]:
def graphs():
    for bidsfile in layout.get(suffix="connectome"):
        with open(bidsfile.path, 'rb') as f:
            data = pickle.load(f)
        np.fill_diagonal(data, 0)
        G = nx.from_numpy_matrix(data)
        yield G

In [ ]:
def betweenness(b_vals, threshold=.5):
    vals = b_vals.values()
    b_range = max(vals) - min(vals)
    margin = b_range*threshold + min(vals)
    above = dict(filter(lambda val: val[1]>margin, b_vals.items()))
    return list(above.keys())
    return len(above) / len(b_vals)

    # b = dict(zip(it.count(), sorted(b_vals)))
    # df = pd.DataFrame({"betweenness": b})
    # return df

In [ ]:
rows = []
for bidsfile in layout.get(suffix="connectome"):
    with open(bidsfile.path, 'rb') as f:
        connectome = pickle.load(f)
    data = copy.deepcopy(connectome.matrix)
    np.fill_diagonal(data, 0)
    G = nx.from_numpy_matrix(data)
    sub = int(bidsfile.entities['subject'])
    cat = categories[sub]
    rows.append({
        "subject": sub,
        "category": cats[cat],
        "degree":np.mean([*zip(*G.degree)][1]),
        "num_regions": len(G.nodes),
        "transitivity": nx.transitivity(G),
        # "efficiency" nx.global_efficiency(G)
    })

df = pd.DataFrame(rows)


In [ ]:
import plotly.express as px

fig = px.violin(
    df,
    x="category",
    color="category",
    y="num_regions",
    points="outliers",
    width=584,
    height=400,
    labels={
        "num_regions": "# Regions",
        "category": "Group"
    },
    title="Number of nodes"
)
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False
)

In [ ]:
rows = []
for bidsfile in layout.get(suffix="connectome", hemi="L"):
    with open(bidsfile.path, 'rb') as f:
        connectome = pickle.load(f)
    data = copy.deepcopy(connectome["connectome"].matrix)
    np.fill_diagonal(data, 0)
    G = nx.from_numpy_matrix(data)
    sub = int(bidsfile.entities['subject'])
    cat = categories[sub]
    for edge in G.edges:
        G.edges[edge]["distance"] = 1/G.edges[edge]["weight"]
    b_vals = nx.betweenness_centrality(G, weight="distance")
    
    sizes = node_sizes_relative(layout.get(subject=bidsfile.entities['subject'], suffix="parcellation", hemi="L")[0].path)
    degrees = degree_sizes_relative(dict(G.degree(weight="weight")))
    for threshold in np.arange(0, 1, 0.1):
        b_nodes = betweenness(b_vals, threshold)
        smallest = min(sizes[i] for i in b_nodes)
        lowest_degree = min(degrees[i] for i in b_nodes)
        
        # weight_above = sum([G.edges[edge]["weight"] for edge in high_edges])
        # total_weight = sum(list(zip(*G.edges(data="weight")))[2])
        rows.append({
            "sub": sub,
            "category": cat,
            "threshold": threshold,    
            "smallest_node": smallest,
            "lowest_degree": lowest_degree
        })

df = pd.DataFrame(rows)
df["category"] = df["category"].astype("uint32")

In [ ]:
import plotly.express as px

grouped = df.groupby(["category", "threshold"]).mean()
dev = df.groupby(["category", "threshold"]).std()
fig = px.line(
    grouped,
    x=grouped.index.get_level_values("threshold"),
   	y="smallest_node",
   	color=grouped.index.get_level_values("category"),
    width=584,
    height=400,
    labels={
        "x": "Betweenness (percentile)",
        "smallest_node": "Smallest Node (percentile)",
        "degree": "Degree",
        "lowest_degree": "Lowest Degree (percentile)",
    },
    title="Minimum size of nodes of given betweenness",
    error_y=dev["lowest_degree"],
    markers=True
)
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False
)

In [ ]:
rows = []
for bidsfile in layout.get(suffix="connectome", hemi="L"):
    with open(bidsfile.path, 'rb') as f:
        connectome = pickle.load(f)
    data = copy.deepcopy(connectome.matrix)
    np.fill_diagonal(data, 0)
    G = nx.from_numpy_matrix(data)
    sub = int(bidsfile.entities['subject'])
    cat = categories[sub]
    for edge in G.edges:
        G.edges[edge]["distance"] = 1/G.edges[edge]["weight"]
    b_vals = nx.betweenness_centrality(G, weight="distance")
    
    sizes = node_sizes(layout.get(subject=bidsfile.entities['subject'], suffix="parcellation", hemi="L")[0].path)
    ddf = pd.concat([
        sizes,
        pd.DataFrame({
            "degree": [d for _, d in G.degree(weight="weight")],
            "betweenness": list(b_vals.values()),
        })
    ], axis=1)
    ddf["subject"] = sub
    ddf["category"] = cat
    rows.append(ddf)

df = pd.concat(rows)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = px.scatter(
    df[df["category"] == 1],
    x="degree",
   	y="betweenness",
    width=584,
    height=400,
    labels={
        "x": "Betweenness (proportion)",
        "smallest_node": "Smallest Node (proportion)",
        "node_size": "Node size",
        "degree": "Degree",
        "betweenness": "Betweenness",
    },
    title="Correlation between degree and betweenness",
    trendline="ols",
    color_discrete_sequence=["darkslategray"],
    opacity=0.6,
    trendline_color_override="blue"
)
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False,
    width=584,
    height=400,

)

In [ ]:
px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared

In [ ]:
def get_spaced_elements(arr, count):
    result = []
    for i in np.round(np.linspace(0, len(arr) - 1, count)).astype(int):
        result.append(arr[i])
    return result

In [ ]:
from collections import defaultdict
num_points = 20
rows = []
for bidsfile in layout.get(suffix="connectome", hemi="L"):
    with open(bidsfile.path, 'rb') as f:
        connectome = pickle.load(f)
    data = copy.deepcopy(connectome["connectome"].matrix)
    np.fill_diagonal(data, 0)
    G = nx.from_numpy_matrix(data)
    sub = int(bidsfile.entities['subject'])
    cat = categories[sub]
    for edge in G.edges:
        G.edges[edge]["distance"] = 1/G.edges[edge]["weight"]

    b_vals = sorted(nx.betweenness_centrality(G, weight="distance").values())
    sizes = sorted(node_sizes(layout.get(subject=bidsfile.entities['subject'], suffix="parcellation", hemi="L")[0].path).values())
    degrees = sorted(dict(G.degree(weight="weight")).values())
    rows.append(
        pd.DataFrame(
            {
                "betweenness": get_spaced_elements(b_vals, num_points),
                "node_size": get_spaced_elements(sizes, num_points),
                "degree": get_spaced_elements(degrees, num_points),
            },
            index=range(num_points),
        ).assign(subject=sub, category=cat)
    )
    
df = pd.concat(rows)

In [ ]:
def hex_to_rgb(hex_color: str) -> tuple:
    hex_color = hex_color.lstrip("#")
    if len(hex_color) == 3:
        hex_color = hex_color * 2
    return int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)

In [ ]:
column = "degree"
gb = (
    df.assign(index=df.index.to_series() / (num_points - 1))
    .groupby(["category", "index"])
)
ddf = pd.concat([gb.mean(), gb.std().rename(lambda n: str(n)+"_std", axis=1)], axis=1)
ddf = ddf.sort_values(column).reset_index()

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

fig = px.line(
    ddf,
    x="index",
    y=column,
    color="category",
    width=584,
    height=400,
    labels={
        "x": "Nodes sorted by increasing betweeness",
        "node_size": "Node Size (# triangles)",
        "betweenness": "Betweenness",
        "degree": "Degree"
    },
    title="Degree distribution",
)
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False
)
for i in ddf["category"].unique():
    dddf = ddf[ddf["category"] == i]

    fig.add_traces([
        go.Scatter(
            x=dddf["index"],
            y=dddf[column+"_std"]+dddf[column],
            mode="lines",
            line=dict(width=0),
        ),
        go.Scatter(
            x=dddf["index"],
            y=dddf[column]-dddf[column+"_std"],
            mode="lines",
            line=dict(width=0),
            fill='tonexty',
            fillcolor=f'rgba{(*hex_to_rgb(px.colors.qualitative.Plotly[i-1]), 0.2)}'
        )
    ])
fig

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

grouped = df.groupby(["category", "point"]).mean()
dev = df.groupby(["category", "point"]).std()
dev_up = grouped + dev
dev_down = grouped - dev
x = grouped.index.get_level_values("point")
color = grouped.index.get_level_values("category")
fig = px.line(
    grouped,
    x=grouped.index.get_level_values("point"),
    y=column,
    color=grouped.index.get_level_values("category"),
    width=584,
    height=400,
    labels={
        "x": "Nodes sorted by increasing betweeness",
        "node_size": "Node Size (# triangles)",
        "degree": "Degree"
    },
    title="Node size distribution",
    error_y=dev[column],
    markers=True
)
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False
)

In [ ]:
rows = []
for bidsfile in layout.get(suffix="connectome", hemi="L"):
    with open(bidsfile.path, 'rb') as f:
        connectome = pickle.load(f)
    data = copy.deepcopy(connectome.matrix)
    np.fill_diagonal(data, 0)
    G = nx.from_numpy_matrix(data)
    sub = int(bidsfile.entities['subject'])
    cat = categories[sub]
    
    size = mesh_size(layout.get(subject=bidsfile.entities['subject'], suffix="parcellation", hemi="L")[0].path)
    total = mesh_size(raw_layout.get(subject=bidsfile.entities['subject'], suffix='smoothwm', hemi="L")[0].path)

    rows.append({
        "sub": sub,
        "category": cats[cat],
        "used": size/total,
        "lost_fibers": connectome.lost_fibers if connectome.lost_fibers > 0 else np.NaN,
        "num_fibers": np.sum(data),
    })

df = pd.DataFrame(rows)

In [ ]:
df

In [ ]:
import plotly.express as px

fig = px.violin(
    df,
    x="category",
    color="category",
    y="num_fibers",
    points="outliers",
    width=584,
    height=400,
    labels={
        "num_regions": "# Regions",
        "category": "Group",
        "used": "proportion of triangles",
        "num_fibers": "Number of fibers",
    },
    title="Number of fibers in connectome"
)
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=False
)